<a href="https://colab.research.google.com/github/mlaricobar/WebScrapingCourse/blob/master/DMC2019I_NOT7_Introduccion_Scrapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a Scrapy

![alt text](https://cdn-images-1.medium.com/max/2400/0*RP3QEulh5aepQ_Ef.png)


Scrapy es un framework de Web Scraping que permite a un desarrollador escribir código para crear un spider, que define cómo una página (o grupo de páginas) será scrapeada.

Una de las características más importantes de Scrapy es que está construida sobre Twisted, una librería de red asíncrona, por lo que Scrapy está implementada de forma no bloqueante (asíncrona) permitiendo así la concurrencia de procesamiento, lo que hace que el rendimiento de scraping sea muy bueno.

Nota: Cuando haces algo de forma sincrónica, esperas a que termine una tarea antes de pasar a otra tarea. Cuando haces algo de forma asíncrona, puedes pasar a otra tarea antes de que finalice.

Scrapy está escrito sobre Python netamente y depende de algunos packages principales de Python:
- lxml
- parsel
- w3lib
- twisted
- cryptography y pyOpenSSL

En mi opinión, esta es una de las herramientas disponibles actualmente en Python que permite manejar tareas complejas de Web Scraping. Puede almacenar en caché las páginas web y agregar paralelismo; solo necesitamos configurar Scrapy correctamente y escribir el código de extracción.

### Comparación contra BeautifulSoup


---

Las 2 herramientas de Web Scraping han sido creadas para diferentes tareas. BeatifulSoup es solamente usado para analizar el código HTML y extraer los datos, por otro lado Scrapy es utilizado para la descarga del HTML, el procesamiento de los datos y su almacenamiento.

Cuando comparamos BeautifulSoup vs Scrapy para descubrir cual es la mejor herramienta para nuestro proyecto, debemos considerar los siguientes factores:


| Framework | BeautifulSoup   | Scrapy   |
|------|------|------|
|   Curva de Aprendizaje  | Muy fácil de aprender, amigable para los principiantes.|La curva de aprendizaje de Scrapy es mucho más pronunciada, necesita comenzar revisando un tutorial o la documentación de Scrapy, y luego trabajar duro para convertirse en un experto en Scrapy.|
|   Ecosistema  | No hay muchos proyectos o plugins relacionados |Muchos proyectos relacionados, plugins disponibles en Github y muchas discusiones en StackOverflow pueden ayudarlo a solucionar un potencial problema.|
|   Extensibilidad  | No es muy fácil ampliar el proyecto. |Puede desarrollar fácilmente un middleware o un pipeline para agregar funciones personalizadas, son fáciles de mantener.|
|   Rendimiento  | Necesita importar el package multiprocessing para que se ejecute más rápido | Muy eficientes, las páginas web se pueden scrapear en poco tiempo, por otro lado, en muchos casos es necesario configurar download_delay para evitar que el spider sea eliminado (banned).|


En resumen, si no tienes mucha experiencia en programación o el trabajo resulta ser muy simple, entonces BeautifulSoup puede ser la mejor opción. Por otro lado, si necesitamos un web scraper más potente y flexible, o si de hecho contamos con algo de experiencia en programación con Python, Scrapy definitivamente es el ganador aquí.

### Instalación

---


Los desarrolladores de Scrapy recomiendan instalar esta herramienta en un entorno virtual. Esta es una buena práctica para tener una versión más limpia de la herramienta; y esto le impide actualizar una dependencia de Scrapy a una versión no compatible, lo que hará que su Scraper no funcione.

Revisar [virtualenvs](https://realpython.com/python-virtual-environments-a-primer/) para más detalle acerca de los entornos virtuales en python.

```
-- Instalación del package virtualenv
pip install virtualenv

-- Creación del un ambiente
cd ~
mkdir Virtualenvs
cd Virtualenvs
virtualenv scrapy_env

-- Activar el ambiente
source scrapy_env/bin/activate

-- Instalación de Scrapy
pip install Scrapy
```

In [0]:
!pip install Scrapy

### Construcción de mi Primer Spider

En este módulo veremos como crear nuestro primer proyecto con Scrapy y nuestro mi primer spider de Scrapy. Adicionalmente, empezaremos a usar y comprender algunos comandos básicos de Scrapy.

#### Comandos de Scrapy

Primero, revisemos brevemente algunos comandos de Scrapy para tener una primera impresión de lo que realizan, y luego podremos aprender un poco más sobre ellos. Por ejemplo escribamos scrapy en el terminal y veamos que obtenemos.

```
$ scrapy
Scrapy 1.6.0 - no active project

Usage:
  scrapy <command> [options] [args]

Available commands:
  bench         Run quick benchmark test
  fetch         Fetch a URL using the Scrapy downloader
  genspider     Generate new spider using pre-defined templates
  runspider     Run a self-contained spider (without creating a project)
  settings      Get settings values
  shell         Interactive scraping console
  startproject  Create new project
  version       Print Scrapy version
  view          Open URL in browser, as seen by Scrapy

  [ more ]      More commands available when run from project directory

Use "scrapy <command> -h" to see more info about a command
```

Como podemos ver, aquí hay una breve lista de comandos de Scrapy. Ahora si queremos ver a detalle algún comando en particular, simplemente usemos `scrapy <comando> -h`. En este módulo, vamos a hacer uso de los comandos `startproject` y `genspider` para empezar a crear un proyecto de Scrapy y un archivo spider. Luego veremos cómo usar los comandos `shell` y `fetch` para poder probar nuestro código.

#### Crear un proyecto Simple

Ahora empecemos a crear un nuevo proyecto desde cero.

```
$ scrapy startproject mi_primer_spider
```
Una vez ejeuctado el comando nos podemos dar cuenta de que nuestro proyecto `mi_primer_spider` ha sido creado. Una vez realizado esto, podemos ver que como salida obtenemos una sugerencia de usar `genspider` para poder gener un spider de Scrapy.

```
You can start your first spider with:
    cd mi_primer_spider
    scrapy genspider example example.com
```

Ejecutemos la sugerencia para ver qué obtenemos.


```

├── scrapy.cfg                # Archivo de configuración de despliegue
└── scrapy_spider             # Modulo de Proyectos, aquí podemos importar nuestro código
    ├── __init__.py
    ├── items.py              # Archivo de definición de los items del proyecto
    ├── middlewares.py        # Archivos de los middlewares del proyecto
    ├── pipelines.py          # Archivo del pipeline del proyecto
    ├── settings.py           # Archivo de configuración del proyecto
    └── spiders               # Directorio donde se encuentran nuestros spiders
        ├── __init__.py
        └── example.py        # Spider que recientemente hemos creado

```

Definición de Conceptos:



1.   **Spiders**: Los Spiders son clases que definen cómo un determinado sitio (o un grupo de sitios) será scrapeado, que comprende el cómo realizar el crawling (es decir, seguir los enlaces) y cómo extraer los datos estructurados de sus páginas (Items). En otras palabras, los spiders son los lugares donde definimos el comportamiento personalizado para scrapear y analizar las páginas de un sitio en particular (o, en algunos casos, un grupo de sitios).
2.   **Items**: Definición de una estructura que contemplará los datos extraídos de la página así como el tratamiento adhoc de los mismos.
3.   **pipelines**: Son clases definidas en Scrapy que permiten procesar los datos extraídos por el Scraping a través de varios componentes que se ejecutan de forma secuencial.



#### Arquitectura de Scrapy

![alt text](https://cdn-images-1.medium.com/max/1600/1*mYXqmVMrbvyp8ZSSkx9lKQ.png)


El flujo de datos en Scrapy está controlado por el motor de ejecución y es de la siguiente forma:



1.   El motor obtiene las solicitudes iniciales desde el Spider para realizar el web crawling.
2.   El motor programa las solicitudes en el Scheduler y solicita las siguientes solicitudes para hacer crawling.
3.   El Scheduler devuelve las siguientes Solicitudes al Motor.
4.   El motor envía las solicitudes al Donwloade, pasando a través de los Donwloader Middlewares.
5.   Una vez que la página termina de descargar, el Downloader genera una Respuesta (con esa página) y la envía al Motor, pasando a través de los Donwloader Middlewares.
6.   El motor recibe la respuesta del Downloader y la envía al Spider para su procesamiento, pasando a través del Spider Middleware.
7.   El Spider procesa la Respuesta y devuelve los elementos extraídos y las nuevas Solicitudes (para seguir) al Motor, pasando a través del Spider Middleware (consulte process_spider_output ()).
8.   El motor envía los elementos procesados a los , luego envía las solicitudes procesadas al programador y solicita las posibles siguientes solicitudes de rastreo.
El proceso se repite (desde el paso 1) hasta que no haya más solicitudes del Programador.

#### Construcción del Spider
Si revisamos nuestro archivo example.py veremos lo siguiente:



```python
# -*- coding: utf-8 -*-
import scrapy


class ExampleSpider(scrapy.Spider):
    name = 'example'
    allowed_domains = ['example.com']
    start_urls = ['http://example.com/']

    def parse(self, response):
        pass

```

Como podemos ver, el archivo spider contiene algunos parámetros, por ejemplo:



1.   **name**: Identifica al Spider. Dentro de un proyecto este debe ser único.
2.   **start_urls**: La lista de URLs semillas, que el spider empezará a realizar web crawling.
3.   **allowed_domains**: Esta configuración es útil para realizar web crawling de una forma más amplia, es decir si el dominio de la URL no está en esta configuración, entonces esa URL será ignorada.
4.   **parse**: Es un método que será llamado para manejar el reponse obtenido por cada petición realizado.


Como ejemplo podemos, modificar nuestro archivo example.py para poder scrapear algunos datos de [la página del catálogo de celulares de Linio](https://www.linio.com.pe/c/celulares-y-tablets).

Es decir podemos cambiarlo de la siguiente forma:

```python
# -*- coding: utf-8 -*-
import scrapy


class LinioSpider(scrapy.Spider):
    name = 'linio_spider'
    allowed_domains = ['linio.com.pe']
    start_urls = ['https://www.linio.com.pe/c/celulares-y-tablets/']

    def parse(self, response):
        product_titles = response.xpath("//div[@id='catalogue-product-container']//div[@class='detail-container']//span[@class='title-section']/text()").extract()
    	  yield {'product_titles': product_titles}

```
Validemos que efectivamente el xpath especificado hace referencia a los títulos de los productos. Una vez que ya tenemos nuestro Spider, podemos ejecutarlo de la siguiente manera:



```
$ scrapy crawl linio_spider
```

Deberíamos ver el resultado del diccionario con la lista de títulos de los productos de la página Linio.

Nota: En caso de obtener respuesta 404 a la petición realizada, descomentar el parámetro `USER_AGENT` en el archivo de `settings.py`

Como ejercicio haremos lo siguiente:

*   Definir otros datos para extraer como la url de la imagen,  el precio y el valor de descuento.
*   Definir la lista de urls a los que se extraerá la información de todos los productos del catálogo de celulares de Linio.



En esta actividad, hemos creado con éxito un Proyecto de Scrapy y un Spider usando algunos comandos de Scrapy. Adicionalmente, hemos desarrollado un spider que nos permita realizar Web Scraping a la página de Linio. 

### Comandos de Scrapy Shell

Scrapy shell nos permite extraer datos y utilizarlo como una herramienta de desarrollo del spider. Se recomienda tener instalado `IPython` antes de usarlo. Para poder usarlo debemos utilizar el comando `scrapy shell`,  luego veremos lo siguiente:



```
2019-05-22 20:18:44 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapy_spider)
[s] Available Scrapy objects:
[s]   scrapy     scrapy module (contains scrapy.Request, scrapy.Selector, etc)
[s]   crawler    <scrapy.crawler.Crawler object at 0x10b5d84e0>
[s]   item       {}
[s]   settings   <scrapy.settings.Settings object at 0x10a0aff28>
[s] Useful shortcuts:
[s]   fetch(url[, redirect=True]) Fetch URL and update local objects (by default, redirects are followed)
[s]   fetch(req)                  Fetch a scrapy.Request and update local objects
[s]   shelp()           Shell help (print this help)
[s]   view(response)    View response in a browser
```

En la mayoría de los casos, la salida es un poco más larga porque otra información, como el middleware y la extensión, también se muestra en el terminal, que se puede ignorar por el momento. En la salida anterior, como puedes ver, hay algunos objetos de Scrapy disponibles en este shell de IPython.

Usaremos el comando fetch para poder realizar Web Crawling a la URL definida por nosotros, por ejemplo continuaremos utilizando la página del catálogo de celulares de Linio.



```python
fetch('https://www.linio.com.pe/c/celulares-y-tablets/')
```

Al revisar el response de la petición, si obtenemos lo siguiente:

```
DEBUG: Crawled (404) <GET https://www.linio.com.pe/c/celulares-y-tablets/> (referer: None
```

nos podemos dar cuenta de que la página bloquea la solicitud que estamos realizando con el framework de Scrapy, y es debido a que el user agent por defecto que utiliza Scrapy es **Scrapy/1.6.0 (+https://scrapy.org)**. Esto lo podemos ver al pintar la variable `settings`.

Para solucionar esto, debemos definir el user agent tal como en el spider creado anteriormente `mi_primer_spider (+http://www.yourdomain.com)` o `Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36`.

Para definir el user agent como header en la petición ejecutar el siguiente código:



```python
url = 'https://www.linio.com.pe/c/celulares-y-tablets/'
request = scrapy.Request(url, headers={'User-Agent': 'mi_primer_spider (+http://www.yourdomain.com)'})
fetch(request)
```

Luego de haber realizado la petición podríamos empezar a revisar el detalle de la página usando el objeto `response`.



```python
reponse.body           #Pintar todo el contenido de la página

response.xpath("//div[@id='catalogue-product-container'])")      #Selector que contiene el elemento con id catalogue-product-container donde se encuentran los datos de los productos.

response.xpath("//div[@id='catalogue-product-container'])").extract()      #Pintar la lista de los elementos que cumplan con el xpath especificado

response.xpath("//div[@id='catalogue-product-container'])").extract_first()      #Pintar el primer elemento que cumpla con el xpath especificado

product_titles = response.xpath("//div[@id='catalogue-product-container']//div[@class='detail-container']//span[@class='title-section']/text()").extract()           #Lista de titulos de productos

img_urls = response.xpath("//div[@id='catalogue-product-container']//div[@class='image-container']//img/@data-lazy").extract()           #Lista de urls de las imágenes de los productos

product_prices = response.xpath("//div[@id='catalogue-product-container']//div[@class='detail-container']//span[@class='price-main']/text()").extract()           #Lista de los precios de los productos

product_discounts = response.xpath("//div[@id='catalogue-product-container']//div[@class='detail-container']//span[@class='discount']/text()").extract()          #Lista de los valores de descuentos de los productos


```

En esta sección hemos visto el uso de Scrapy Shell que nos permitirá desarrollar y hacer pruebas rápidamente sobre los Spiders que queremos definir.



Otra forma de como extraer los valores de los productos es de la siguiente forma:



```python
# -*- coding: utf-8 -*-
import scrapy
from scrapy.selector import Selector

class ExampleSpider(scrapy.Spider):
    name = 'linio_spider'
    allowed_domains = ['linio.com.pe']
    start_urls = ['https://www.linio.com.pe/c/celulares-y-tablets/']
    #start_urls = ["https://www.linio.com.pe/c/celulares-y-tablets?page={0}".format(i) for i in range(1,18)]

    def parse(self, response):
    	product_list = response.xpath("//div[@id='catalogue-product-container']//div[@class='catalogue-product row']")#.extract()

    	for product_selector in product_list:
    		#product_selector = Selector(text=product)
    		yield {
    		"title": product_selector.xpath(".//span[@class='title-section']/text()").extract_first(),
    		"img_url": product_selector.xpath(".//div[@class='image-container']//img/@data-lazy").extract_first(),
    		"product_price": product_selector.xpath(".//div[@class='detail-container']//span[@class='price-main']/text()").extract_first(),
    		"product_discount": product_selector.xpath(".//div[@class='detail-container']//span[@class='discount']/text()").extract_first()
    		}
```

Nota: Si se han dado cuenta en el xpath de cada campo hemos añadido el . al inicio para que realice la busqueda a nivel de selector del loop, sino lo hará a nivel del documento.

### Paginación

Una forma de realizar web crawling es iterar sobre una lista de urls, en caso de que se de la casuísitca de no tener un patrón deberíamos iterar sobre un paginado. En esta sección veremos como implementarlo en la página de Linio.



```python
next_page = response.xpath("//nav[@class='pagination-container']//a[@class='page-link page-link-icon']/@href").extract()[-2]  #Obtiene la url del paginado
    	if next_page is not None:
    		next_page_link= response.urljoin(next_page)
    		yield scrapy.Request(url=next_page_link, callback=self.parse)
```



Ejecutar el Spider

Para ejecutar el spider y guardar los datos debemos ejecutar lo siguiente:



```
scrapy crawl linio_spider -o data.json
```

Si hasta el momento revisamos los datos obtenidos nos podemos dar cuenta de que algunos necesitan limpieza por ejemplo el precio del producto.

![alt text](https://github.com/mikolarico/course-images/blob/master/Screen%20Shot%202019-05-24%20at%2015.20.15.png?raw=true)



Un enfoque de limpieza sería el cargar nuestros datos a un dataframe y luego aplicarle operaciones de limpieza. Revisar el otro notebook de Exploración.


Sin embargo, Scrapy proporciona una forma de cómo poder realizar la limpieza de los datos extraídos a través del uso de los `Items`.

### Items


Los Spiders de Scrapy pueden devolver los datos extraídos como diccionarios de Python. Si bien son convenientes y familiares, los diccionarios de Python carecen de estructura: es fácil de obtener un error por el tipo en un nombre de campo o de obtener datos inconsistentes, especialmente en un proyecto grande con muchos Spiders.


Para definir un formato de datos comun Scrapy proporciona la clase `Item`. Los objetos de la clase `Item` son contenedores simples usados para coleccionar los datos scrapeados. Ellos proporcionan un API (casi con el formato de diccionario) con una sintaxis conveniente para declarar sus campos o atributos.

En el archivo items.py definir lo siguiente:



```python
# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://doc.scrapy.org/en/latest/topics/items.html

import scrapy
import re
from scrapy.loader.processors import MapCompose, TakeFirst


def remove_whitespace(value):
	return value.strip()

def set_prefix_url(value):
	return 'https' + value

def clean_price_value(value):
	return float(re.search(r'S/\s(.+)', value).group(1).replace(",", ""))

def clean_discount_value(value):
	return int(re.search(r'\d+', value).group())



class MiPrimerSpiderItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    title = scrapy.Field(
    	input_processor = MapCompose(remove_whitespace, set_prefix_url), 
    	output_processor=TakeFirst()
    	)

    img_url = scrapy.Field(output_processor=TakeFirst())
    product_price = scrapy.Field(input_processor=MapCompose(remove_whitespace, clean_price_value), output_processor=TakeFirst())
    product_discount = scrapy.Field(input_processor=MapCompose(remove_whitespace, clean_discount_value), output_processor=TakeFirst())
    url_pagination = scrapy.Field(output_processor=TakeFirst())

```

y en el archivo example.py:

```python
# -*- coding: utf-8 -*-
import scrapy
from mi_primer_spider.items import MiPrimerSpiderItem
from scrapy.loader import ItemLoader

class ExampleSpider(scrapy.Spider):
    name = 'linio_spider'
    allowed_domains = ['linio.com.pe']
    start_urls = ['https://www.linio.com.pe/c/celulares-y-tablets/']
    #start_urls = ["https://www.linio.com.pe/c/celulares-y-tablets?page={0}".format(i) for i in range(1,18)]

    def parse(self, response):


    	product_list = response.xpath("//div[@id='catalogue-product-container']//div[@class='catalogue-product row']")#.extract()

    	for product_selector in product_list:
    		#product_selector = Selector(text=product)
    		l = ItemLoader(item=MiPrimerSpiderItem(), selector=product_selector)
    		l.add_xpath("title", ".//span[@class='title-section']/text()")
    		l.add_xpath("img_url", ".//div[@class='image-container']//img/@data-lazy")
    		l.add_xpath("product_price", ".//div[@class='detail-container']//span[@class='price-main']/text()")
    		l.add_xpath("product_discount", ".//div[@class='detail-container']//span[@class='discount']/text()")
    		l.add_value("url_pagination", response.request.url)
    		yield l.load_item()

    	next_page = response.xpath("//nav[@class='pagination-container']//a[@class='page-link page-link-icon']/@href").extract()[-2]
    	if next_page is not None:
    		next_page_link= response.urljoin(next_page)
    		yield scrapy.Request(url=next_page_link, callback=self.parse)

    		#data_list.append(data_row)
    	#yield {"data": data_list}
      
```




### Ejecución en Google Colab

In [0]:
!scrapy

Scrapy 1.6.0 - no active project

Usage:
  scrapy <command> [options] [args]

Available commands:
  bench         Run quick benchmark test
  fetch         Fetch a URL using the Scrapy downloader
  genspider     Generate new spider using pre-defined templates
  runspider     Run a self-contained spider (without creating a project)
  settings      Get settings values
  shell         Interactive scraping console
  startproject  Create new project
  version       Print Scrapy version
  view          Open URL in browser, as seen by Scrapy

  [ more ]      More commands available when run from project directory

Use "scrapy <command> -h" to see more info about a command


In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
!ls -l /gdrive/'My Drive'/Work/DMC/'01. Web Scraping'/'Clase 04'/mi_primer_spider

total 35
-rw------- 1 root root 31195 May 30 20:06 DMC2019I_NOT7_Pipeline_Limpieza.ipynb
drwx------ 4 root root  4096 May 30 20:06 mi_primer_spider
-rw------- 1 root root   275 May 30 20:06 scrapy.cfg


In [0]:
!cp -r /gdrive/'My Drive'/Work/DMC/'01. Web Scraping'/'Clase 04'/mi_primer_spider mi_primer_spider

In [0]:
%cd /content/mi_primer_spider

/content/mi_primer_spider


In [0]:
!ls -l

total 40
-rw------- 1 root root 31195 May 30 20:12 DMC2019I_NOT7_Pipeline_Limpieza.ipynb
drwx------ 4 root root  4096 May 30 20:12 mi_primer_spider
-rw------- 1 root root   275 May 30 20:12 scrapy.cfg


In [0]:
!scrapy crawl linio_spider_v2 -o data_v2.json

In [0]:
!ls -l

total 5392
-rw-r--r-- 1 root root 5480272 May 30 20:19 data_v2.json
-rw------- 1 root root   31195 May 30 20:12 DMC2019I_NOT7_Pipeline_Limpieza.ipynb
drwx------ 4 root root    4096 May 30 20:12 mi_primer_spider
-rw------- 1 root root     275 May 30 20:12 scrapy.cfg


In [0]:
import pandas as pd

In [0]:
df = pd.read_json("data_v2.json")

In [0]:
df.shape

(13892, 10)

In [0]:
df.head()

,availability,brand,category,image,model,price,priceCurrency,sku,title,url
0,NaN,Samsung,Ultra HD / 4K TV,//i.linio.com/p/2640370e05d49b2a67c1b3f1389592...,UN58NU7100G,1599.00,PEN,SA026EL0AM8GELPE,"Tv led Smart Samsung 58"" UN58NU7100G Ultra HD ...",/p/tv-led-smart-samsung-58-un58nu7100g-ultra-h...
1,NaN,Apple,Desbloqueados,//i.linio.com/p/a5f87f2fc4d52a078e17a100425d0c...,iPhone 8 Plus,1819.00,PEN,AP032EL043GG6LPE,Apple iPhone 8 Plus 64GB - Space Gray,/p/apple-iphone-8-plus-64gb-space-gray-ymcw0x
2,NaN,National,Cocinas,//i.linio.com/p/533c757f0144d1efb31564baacacf2...,Pro 2400 W 01 Hornilla,89.99,PEN,NA979HL0MHEP2LPE,Cocina De Induccion LED Digital National Pro 2...,/p/cocina-de-induccion-led-digital-national-pr...
3,NaN,Go Pro,Cámaras Acuáticas,//i.linio.com/p/31379c2ce463b3da6be70014121f2a...,CHDHX-502,715.00,PEN,GO903EL0F6XDSLPE,"Go Pro Hero 5, Negro, 1x12 Megapixel CMOS Sens...",/p/go-pro-hero-5-negro-1x12-megapixel-cmos-sen...
4,NaN,CL IMPORT,Cepillos y peines,//i.linio.com/p/9d536da8100c2d1dd8e88a9730368e...,,28.00,PEN,CL708HB1A71XSLPE,Ultima Version Cepillo Alisador Straight Artif...,/p/ultima-version-cepillo-alisador-straight-ar...


In [0]:
df["url"].nunique()

13892

In [0]:
!cp data_v2.json /gdrive/'My Drive'/Work/DMC/'01. Web Scraping'/'Clase 04'/mi_primer_spider/data_v2.json

### Item Pipeline
Despues que un Item haya sido *scrapeado* por un Spider, será enviado a un Pipeline para que sea procesado por varios componentes. Cada Pipeline es una clase de Python que implementa un método simple. El componente puede realizar una acción sobre el Item, modificarlo, eliminarlo, o enviarlo a otro componente.

El Pipeline del Item puede ser usado para validar los datos scrapeados, revisar el duplicado de datos, o insertar los datos en bases de datos tales como MySQL, PostgreSQL, CosmosDB o MongoDB.

En este pequeño módulo, construiremos un pipeline que permita almacenar los datos scrapeados en una Base de Datos como Azure Cosmos DB.


**En el archivo Settings.py:**
En el archivo de configuraciones debemos definir las variables de cadenas de conexión que utilizaremos dentro del Pipeline.



```python
# Adhoc settings
ENDPOINT = "##### < PASTE YOUR ENDPOINT HERE > #####"
MASTERKEY = "##### < PASTE YOUR MASTER KEY HERE > #####"

COLLECTION_LINK =  "##### < PASTE YOUR COLLECTION KEY HERE > #####"


ITEM_PIPELINES = {
    'linio_project_v2.pipelines.ProductPipeline': 300,
}


```





**En el archivo Pipeline.py**: Aquí debemos definir la clase del Pipeline así como la lógica que se implementará por cada Item. 



```python
# -*- coding: utf-8 -*-

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://doc.scrapy.org/en/latest/topics/item-pipeline.html

from pydocumentdb import documents
from pydocumentdb import document_client

from scrapy.utils.project import get_project_settings as get_set


class ProductPipeline(object):
	def __init__(self):
		"""
		Inicia el cliente para conectarnos a la BD de Cosmos.
		"""
		connectionPolicy = documents.ConnectionPolicy()
		connectionPolicy.EnableEndpointDiscovery
		connectionPolicy.PreferredLocations = ["East US 2"]

		self.client = document_client.DocumentClient(get_set().get("ENDPOINT"), {'masterKey': get_set().get("MASTERKEY")}, connectionPolicy)

	def process_item(self, item, spider):
		d = dict(item)
		try:
			self.client.CreateDocument(get_set().get("COLLECTION_LINK"), d)
		except:
			raise

		return item
```



Al ejecutar el spider, se valida que efectivamente los registros se han ejecutado:

![alt text](https://github.com/mikolarico/course-images/blob/master/product_in_cosmos.png?raw=true)

### Despliegue en Producción

Existen diferentes formas de cómo desplegar los spiders de Scrapy que hemos desarrollado para su ejecución periódica o regular. Por lo general ejecutar los spiders de Scrapy en una máquina local es muy conveniente para la etapa de desarrollo, pero no tanto cuando se necesita ejecutar Spiders de larga ejecución o migrar Spiders para que se ejecuten en ambientes de producción continuamente. Aquí es donde entran las soluciones de despliegue de Scrapy.

#### Scrapy Cloud

Es un servicio en nube y proporcionado por ScrapingHub, la compañía creadora de Scrapy. Con Scrapy Cloud nos evitamos la necesidad de configurar y monitorear servidores ya que proporciona una interfaz de usuario agradable para administrar los Spiders y revisar los ítems, registros y estadísticas resultados del Scraping.

Para implementar los Spiders en Scrapy Cloud, podemos usar la herramienta de línea de comandos **shub**. Para ello necesitamos instalarla con el siguiente comando:



```
pip install shub
```

El siguiente paso es desplegar nuestro proyecto de Scrapy en Scrapy Cloud. Para ello, necesitamos el **API key** y el **numeric ID** del proyecto. Podemos encontrar esos dos datos en la página de **Code & Deploys**. Una vez que tengamos esos datos a la mano, debemos ejecutar el siguiente comando

```
shub login
```
Con este comando guardamos nuestro **API key** en un archivo local (~/scrapinghub.yml). Si queremos eliminarlo usemos el comando:

```
shub logout
```

Una vez logueados, ubíquemonos en nuestro proyecto y ejecutemos el siguiente comando para desplegar nuestro proyecto a Scrapy Cloud:


```
shub deploy
```

Una vez que realizamos el despliegue, podemos ejecutar el spider por medio del siguiente comando:


```
shub schedule linio_spider
```





### Integración con Splash

**Instalación de Splash:**

La recomendación de instalar **Splash** es por medio de Docker. Referencias:

*   [Instalación y Configuración de docker](https://docs.docker.com/toolbox/toolbox_install_windows/)
*   [Configuración de Splash](https://blog.scrapinghub.com/2015/03/02/handling-javascript-in-scrapy-with-splash)





### Integración con Selenium

Para poder realizar la integración de Selenium con Scrapy, debemos utilizar el código de Selenium en el Downloader Middleware del Proyecto. 

Debemos recordar que los métodos de esta clase se llaman siempre que Scrapy realiza una solicitud. Modifica la solicitud / respuesta de alguna manera y la devuelve a Scrapy.


### Notas

In [0]:
import requests
# importar la librería BeautifulSoup
from bs4 import BeautifulSoup

In [0]:
response = requests.get('https://www.linio.com.pe/c/celulares-y-tablets/')
soup = BeautifulSoup(markup=response.content, features='html.parser')

In [0]:
soup.find(name="div", attrs={"id": "catalogue-product-container"})

In [0]:
!wget -U 'mi_primer_spider (+http://www.yourdomain.com)' https://www.linio.com.pe/c/celulares-y-tablets/

In [0]:
!wget -U 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36' https://www.linio.com.pe/c/celulares-y-tablets/